# Test computation of background probablitily

This notbook compares the computation of the source background probability, used for stacking clusters
(look at `demo_compute_deltasigma_weights.ipynb` for more details).

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import scipy

import clmm
from clmm import Cosmology
from clmm import GalaxyCluster
from clmm.dataops import compute_galaxy_weights
from clmm.support import mock_data as mock

clmm.__version__

- Creating the same mock catalog with three different ways of storing the photo-z PDF information (see `demo_mock_cluster.ipynb` for details):

In [ ]:
cosmo = Cosmology(H0=71.0, Omega_dm0=0.265 - 0.0448, Omega_b0=0.0448, Omega_k0=0.0)
cluster_z = 0.4
args = (
    1e14,  # cluster_mass
    cluster_z,
    4,  # concentration
    cosmo,
)
kwargs = dict(
    zsrc="chang13",
    delta_so=200,
    massdef="critical",
    halo_profile_model="nfw",
    zsrc_min=0.0,
    zsrc_max=3.0,
    field_size=10.0,
    shapenoise=0.5,
    photoz_sigma_unscaled=0.05,
    mean_e_err=0.1,
    ngals=10000,
)

np.random.seed(41363)
noisy_data_z_sb = mock.generate_galaxy_catalog(*args, **kwargs, pzpdf_type="shared_bins")
np.random.seed(41363)
noisy_data_z_ib = mock.generate_galaxy_catalog(*args, **kwargs, pzpdf_type="individual_bins")
np.random.seed(41363)
noisy_data_z_qt = mock.generate_galaxy_catalog(*args, **kwargs, pzpdf_type="quantiles")

In [ ]:
label = "shared bins"
for i, data in enumerate(noisy_data_z_sb[:5]):
    plt.plot(noisy_data_z_sb.pzpdf_info["zbins"], data["pzpdf"], lw=0.5, color=f"C{i}", label=label)
    label = None

label = "individual bins"
for i, data in enumerate(noisy_data_z_ib[:5]):
    plt.plot(data["pzbins"], data["pzpdf"], lw=0.9, color=f"C{i}", ls="--", label=label)
    label = None


pzbins, pzpdfs = noisy_data_z_qt.get_pzpdfs()
label = "quantiles"
for i, data in enumerate(pzpdfs[:5]):
    plt.plot(pzbins, data, lw=3, color=f"C{i}", ls=":", label=label)
    label = None
plt.xlim(0.4, 2.1)
plt.legend()
plt.show()

## Test P(z) integrals

Probability to be in the background of the cluster i.e. to be higher than a given threshold is given by:
$$ 
P(z > z_l) = \int_{z_l}^{+\infty} dz\ p(z) 
$$
,

where `z_l` is the redshift of the lens.

Below we will compare the performance of this computation with `clmm` and with `qp` considering the different types of photo-z PDF data.

### From CLMM

In `clmm`, the integration of the PDF is made with the `_integ_pzfuncs` function:

In [ ]:
from clmm.dataops import _integ_pzfuncs

In [ ]:
integrals = {
    "clmm_shared": _integ_pzfuncs(*noisy_data_z_sb.get_pzpdfs()[::-1], cluster_z),
    "clmm_individual": _integ_pzfuncs(*noisy_data_z_ib.get_pzpdfs()[::-1], cluster_z),
    "clmm_quantiles": _integ_pzfuncs(*noisy_data_z_qt.get_pzpdfs()[::-1], cluster_z),
}

### From `qp`

In `qp`, this integral can be done using the cdf of a data ensemble.
Below we present how to make the corresponding ensemble for each type of PDF data stored:

In [ ]:
import qp

In [ ]:
qp_dat = qp.Ensemble(
    qp.interp,
    data={"xvals": noisy_data_z_sb.pzpdf_info["zbins"], "yvals": noisy_data_z_sb["pzpdf"]},
)
integrals["qp_shared"] = 1 - qp_dat.cdf(cluster_z)[:, 0]

In [ ]:
qp_dat2 = qp.Ensemble(
    qp.interp_irregular,
    data={"xvals": noisy_data_z_ib["pzbins"], "yvals": noisy_data_z_ib["pzpdf"]},
)
integrals["qp_individual"] = 1 - qp_dat2.cdf(cluster_z)[:, 0]

In [ ]:
qp_dat3 = qp.Ensemble(
    qp.quant,
    data={
        "locs": noisy_data_z_qt["pzquantiles"],
        "quants": noisy_data_z_qt.pzpdf_info["quantiles"],
    },
)
integrals["qp_quantiles"] = 1 - qp_dat3.cdf(cluster_z)[:, 0]

### True Values

Comparison of these integrals with the analytical form.
For a gaussian distribution, the integral can be computed with the error function:

$$
P(z > z_l) = 
\frac{1}{\sqrt{2\pi\sigma_z^2}}\int_{z_{l}}^{+\infty} dz\ e^{-\frac{(z-z_{gal})^2}{2\sigma_z^2}} =
\frac{1}{2} {\rm erfc}\left(\frac{z_{l}-z_{gal}}{\sqrt{2}\sigma_z}\right)
$$

In [ ]:
from scipy.special import erfc

In [ ]:
true_integ = 0.5 * erfc(
    (cluster_z - noisy_data_z_sb["z"]) / (0.05 * (1 + noisy_data_z_sb["z"]) * np.sqrt(2))
)

### Difference

Show relative difference to the analytical form:

In [ ]:
from scipy.stats import binned_statistic

In [ ]:
fig, axes = plt.subplots(2, sharex=True, sharey=True, figsize=(7, 5))
bins = np.linspace(0, 1, 21)
for comp_case, ax in zip(("clmm", "qp"), axes):
    for i, pdf_case in enumerate(("shared", "individual", "quantiles")):
        dx = (i - 1) * 0.01
        integ = integrals[f"{comp_case}_{pdf_case}"]
        ax.errorbar(
            0.5 * (bins[1:] + bins[:-1]) + dx,
            binned_statistic(integ, (integ / true_integ - 1) * 100, bins=bins)[0],
            binned_statistic(integ, (integ / true_integ - 1) * 100, bins=bins, statistic="std")[0],
            label=pdf_case,
            lw=0.7,
        )
    ax.axhline(0, c="0", ls="--", lw=0.5)
    ax.minorticks_on()
    ax.grid()
    ax.grid(which="minor", lw=0.3)
    ax.set_ylim(-20, 30)
    ax.set_title(f"{comp_case} integral")
axes[0].legend()
axes[1].set_xlabel("True integral")
for ax in axes:
    ax.set_ylabel("rel. diff [%]")